In [51]:
!pip install biopython==1.77

     |████████████████████████████████| 2.3 MB 1.2 MB/s eta 0:00:01
  Attempting uninstall: biopython
    Found existing installation: biopython 1.78
    Uninstalling biopython-1.78:
      Successfully uninstalled biopython-1.78


In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from collections import defaultdict

In [2]:
# read names and postions from narrowPeak file
positions = defaultdict(list)
with open('curax(14h)-Z22-vs-IgG-[FLAG-ZBP1-PLUS].narrowPeak') as f:
    for line in f:
        chrom, start, stop, name, score, strand, signalValue, pValue, qValue, peak = line.split()
        positions[chrom].append((int(start), int(stop), int(score), float(signalValue), float(pValue), float(qValue), int(peak), '14h'))
with open('curax(10h-14h)-Z22-vs-IgG-[FLAG-ZBP1-MINUS].narrowPeak') as f:
    for line in f:
        chrom, start, stop, name, score, strand, signalValue, pValue, qValue, peak = line.split()
        positions[chrom].append((int(start), int(stop), int(score), float(signalValue), float(pValue), float(qValue), int(peak), '10h-14h'))        

In [3]:
# search for short sequences
short_seq_records = []
f = open('train_data.csv', 'w') 
for chrom in positions:
    print(chrom)
    long_seq_record = SeqIO.to_dict(SeqIO.parse(open(chrom + '.fa'), 'fasta'))[chrom]
#     print(long_seq_record)
    for (start, stop, score, signalValue, pValue, qValue, peak, file) in positions[chrom]:
        long_seq = long_seq_record.seq
        alphabet = long_seq.alphabet
        short_seq = str(long_seq)[start-1:stop]
        short_seq_record = SeqRecord(Seq(short_seq, alphabet), id=chrom, description='')
        short_seq_records.append(short_seq_record)
#         print(short_seq)
        with open('train_data.csv', 'a') as f:
            f.write(','.join([chrom, str(score), str(signalValue), str(pValue), str(qValue), str(peak), file, short_seq]))
            f.write('\n')


chr1
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr1_GL456211_random
chr1_GL456221_random
chr2
chr3
chr4
chr5
chr5_GL456354_random
chr6
chr7
chr8
chr9
chrM
chrUn_GL456370
chrUn_GL456383
chrUn_GL456390
chrUn_GL456392
chrUn_GL456393
chrUn_GL456396
chrUn_JH584304
chrX
chrY
chr4_GL456216_random
chr4_JH584293_random
chr4_JH584294_random
chrUn_GL456378
chrUn_GL456389


In [4]:
import pandas as pd
file3 = open("train_data.csv")
df = pd.read_csv('train_data.csv', delimiter=',',
                names=['chrom', 'score', 'signalValue', 'pValue', 'qValue', 'peak', 'file', 'short_seq'])
df.head()
# print(file3.read())

,chrom,score,signalValue,pValue,qValue,peak,file,short_seq
0,chr1,84,4.19611,12.50863,8.41826,176,14h,cctgccgaacttaggaaattagtctgaacaggtgagagggtgcgcc...
1,chr1,29,4.12410,6.59635,2.90590,139,14h,CTCTGGCCGGTGTTTGCCATGCTCAGGCCGGTGTTTGCCATGCTCA...
2,chr1,107,2.47923,14.95097,10.76389,494,14h,gcaaatctccaccaaattggattcacctgtatcctggtacacctgc...
3,chr1,86,5.25893,12.75605,8.65332,231,14h,agaacacaaatgccagcccaggctactatacccggccaaactctca...
4,chr1,82,4.93616,12.34533,8.26059,224,14h,acggtcacccgtgcagcctgccctccgcggagtcccggagccaaga...


In [6]:
#generating lengths for negative class  
import numpy as np
from sklearn.neighbors import KernelDensity
X = []
for chrom in positions:
    for (start, stop, score, signalValue, pValue, qValue, peak, file) in positions[chrom]:
        X.append(stop - start)

Y = np.array(X).reshape(-1, 1)
kde = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(Y)
result = kde.sample(n_samples=10)
for i in result:
    print(int(i[0]))

320
591
343
424
538
285
610
405
412
561


In [10]:
for chrom in positions:
    long_seq_record = SeqIO.to_dict(SeqIO.parse(open(chrom + '.fa'), 'fasta'))[chrom]
    print(len(long_seq_record.seq))

195471971
130694993
122082543
120129022
120421639
124902244
104043685
98207768
94987271
90702639
61431566
241735
206961
182113224
160039680
156508116
151834684
195993
149736546
145441459
129401213
124595110
16299
26764
38659
24668
23629
55711
21240
114452
171031299
91744698
66673
207968
191905
31602
28772
